In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null


In [ ]:
!wget https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar -xvf spark-3.4.0-bin-hadoop3.tgz


--2025-07-15 14:35:12--  https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388407094 (370M) [application/x-gzip]
Saving to: ‘spark-3.4.0-bin-hadoop3.tgz’

spark-3.4.0-bin-had 100%[===================>] 370.41M   572KB/s    in 20m 15s 

2025-07-15 14:55:28 (312 KB/s) - ‘spark-3.4.0-bin-hadoop3.tgz’ saved [388407094/388407094]

spark-3.4.0-bin-hadoop3/
spark-3.4.0-bin-hadoop3/data/
spark-3.4.0-bin-hadoop3/data/streaming/
spark-3.4.0-bin-hadoop3/data/streaming/AFINN-111.txt
spark-3.4.0-bin-hadoop3/data/graphx/
spark-3.4.0-bin-hadoop3/data/graphx/followers.txt
spark-3.4.0-bin-hadoop3/data/graphx/users.txt
spark-3.4.0-bin-hadoop3/data/mllib/
spark-3.4.0-bin-hadoop3/data/mllib/als/
spark-3.4.0-bin-hadoop3/data/mllib/als/test.data
spark-

In [ ]:
!pip install -q findspark

In [ ]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

findspark.init()

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving threat.csv to threat.csv


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ThreatDetection").getOrCreate()

In [ ]:
df = spark.read.csv("threat.csv", header=True, inferSchema=True)

In [ ]:
print("Number of columns:", len(df.columns))
print("Columns:")
for column in df.columns:
  print(column)

Number of columns: 79
Columns:
 Destination Port
 Flow Duration
 Total Fwd Packets
 Total Backward Packets
Total Length of Fwd Packets
 Total Length of Bwd Packets
 Fwd Packet Length Max
 Fwd Packet Length Min
 Fwd Packet Length Mean
 Fwd Packet Length Std
Bwd Packet Length Max
 Bwd Packet Length Min
 Bwd Packet Length Mean
 Bwd Packet Length Std
Flow Bytes/s
 Flow Packets/s
 Flow IAT Mean
 Flow IAT Std
 Flow IAT Max
 Flow IAT Min
Fwd IAT Total
 Fwd IAT Mean
 Fwd IAT Std
 Fwd IAT Max
 Fwd IAT Min
Bwd IAT Total
 Bwd IAT Mean
 Bwd IAT Std
 Bwd IAT Max
 Bwd IAT Min
Fwd PSH Flags
 Bwd PSH Flags
 Fwd URG Flags
 Bwd URG Flags
 Fwd Header Length
 Bwd Header Length
Fwd Packets/s
 Bwd Packets/s
 Min Packet Length
 Max Packet Length
 Packet Length Mean
 Packet Length Std
 Packet Length Variance
FIN Flag Count
 SYN Flag Count
 RST Flag Count
 PSH Flag Count
 ACK Flag Count
 URG Flag Count
 CWE Flag Count
 ECE Flag Count
 Down/Up Ratio
 Average Packet Size
 Avg Fwd Segment Size
 Avg Bwd Segment Si

In [ ]:
df.select(" Label").distinct().show(truncate=False)


+------------+
| Label      |
+------------+
|Infiltration|
|Web Attack  |
|DoS         |
|Benign      |
|Heartbleed  |
+------------+



In [ ]:
df.groupBy(" Label").count().orderBy("count", ascending=False).show(truncate=False)


+------------+-----+
| Label      |count|
+------------+-----+
|Benign      |5000 |
|DoS         |2500 |
|Web Attack  |1500 |
|Infiltration|30   |
|Heartbleed  |9    |
+------------+-----+



In [ ]:
df = df.toDF(*[col.strip() for col in df.columns])

In [ ]:
print(df.columns)

['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count

In [ ]:
df.groupBy("Label").count().orderBy("count", ascending=False).show(truncate=False)

+------------+-----+
|Label       |count|
+------------+-----+
|Benign      |5000 |
|DoS         |2500 |
|Web Attack  |1500 |
|Infiltration|30   |
|Heartbleed  |9    |
+------------+-----+



In [ ]:
df = df.filter(~df.Label.isin("Infiltration", "Heartbleed"))

In [ ]:
df.groupBy("Label").count().show()

+----------+-----+
|     Label|count|
+----------+-----+
|Web Attack| 1500|
|       DoS| 2500|
|    Benign| 5000|
+----------+-----+



In [ ]:
df.show(5)

+----------------+-------------+-----------------+----------------------+---------------------------+---------------------------+---------------------+---------------------+----------------------+---------------------+---------------------+---------------------+----------------------+---------------------+------------+--------------+-------------+------------+------------+------------+-------------+------------+-----------+-----------+-----------+-------------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+-----------------+-----------------+-------------+-------------+-----------------+-----------------+------------------+-----------------+----------------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-------------+-------------------+--------------------+--------------------+-------------------+------------------+--------------------+--------------

In [ ]:
df.printSchema()

root
 |-- Destination Port: integer (nullable = true)
 |-- Flow Duration: integer (nullable = true)
 |-- Total Fwd Packets: integer (nullable = true)
 |-- Total Backward Packets: integer (nullable = true)
 |-- Total Length of Fwd Packets: integer (nullable = true)
 |-- Total Length of Bwd Packets: integer (nullable = true)
 |-- Fwd Packet Length Max: integer (nullable = true)
 |-- Fwd Packet Length Min: integer (nullable = true)
 |-- Fwd Packet Length Mean: double (nullable = true)
 |-- Fwd Packet Length Std: double (nullable = true)
 |-- Bwd Packet Length Max: integer (nullable = true)
 |-- Bwd Packet Length Min: integer (nullable = true)
 |-- Bwd Packet Length Mean: double (nullable = true)
 |-- Bwd Packet Length Std: double (nullable = true)
 |-- Flow Bytes/s: string (nullable = true)
 |-- Flow Packets/s: string (nullable = true)
 |-- Flow IAT Mean: double (nullable = true)
 |-- Flow IAT Std: double (nullable = true)
 |-- Flow IAT Max: integer (nullable = true)
 |-- Flow IAT Min: in

In [ ]:
from pyspark.sql.functions import col

df = df.withColumn("Flow Bytes/s", col("Flow Bytes/s").cast("double"))
df = df.withColumn("Flow Packets/s", col("Flow Packets/s").cast("double"))

In [ ]:
df.printSchema()

root
 |-- Destination Port: integer (nullable = true)
 |-- Flow Duration: integer (nullable = true)
 |-- Total Fwd Packets: integer (nullable = true)
 |-- Total Backward Packets: integer (nullable = true)
 |-- Total Length of Fwd Packets: integer (nullable = true)
 |-- Total Length of Bwd Packets: integer (nullable = true)
 |-- Fwd Packet Length Max: integer (nullable = true)
 |-- Fwd Packet Length Min: integer (nullable = true)
 |-- Fwd Packet Length Mean: double (nullable = true)
 |-- Fwd Packet Length Std: double (nullable = true)
 |-- Bwd Packet Length Max: integer (nullable = true)
 |-- Bwd Packet Length Min: integer (nullable = true)
 |-- Bwd Packet Length Mean: double (nullable = true)
 |-- Bwd Packet Length Std: double (nullable = true)
 |-- Flow Bytes/s: double (nullable = true)
 |-- Flow Packets/s: double (nullable = true)
 |-- Flow IAT Mean: double (nullable = true)
 |-- Flow IAT Std: double (nullable = true)
 |-- Flow IAT Max: integer (nullable = true)
 |-- Flow IAT Min: in

In [ ]:
df.select("Flow Bytes/s", "Flow Packets/s").summary().show()


+-------+------------+--------------+
|summary|Flow Bytes/s|Flow Packets/s|
+-------+------------+--------------+
|  count|        9000|          9000|
|   mean|    Infinity|      Infinity|
| stddev|         NaN|           NaN|
|    min|      -1.2E7|    -2000000.0|
|    25%|  9.98029936|   0.302101249|
|    50%| 143.4110016|   2.991603566|
|    75%| 9591.104887|   402.8197382|
|    max|    Infinity|      Infinity|
+-------+------------+--------------+



In [ ]:
from pyspark.sql.functions import col, isnan, isnull, when, sum as spark_sum

numeric_cols = [name for name, dtype in df.dtypes if dtype in ('double', 'int')]

stats = df.select([
    spark_sum(
        when(
            isnan(col(f"`{c}`")) | isnull(col(f"`{c}`")) | (col(f"`{c}`") == float('inf')) | (col(f"`{c}`") == float('-inf')),
            1
        ).otherwise(0)
    ).alias(c)
    for c in numeric_cols
])

stats.show(truncate=False)


+----------------+-------------+-----------------+----------------------+---------------------------+---------------------------+---------------------+---------------------+----------------------+---------------------+---------------------+---------------------+----------------------+---------------------+------------+--------------+-------------+------------+------------+------------+-------------+------------+-----------+-----------+-----------+-------------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+-----------------+-----------------+-------------+-------------+-----------------+-----------------+------------------+-----------------+----------------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-------------+-------------------+--------------------+--------------------+-------------------+------------------+--------------------+--------------

In [ ]:
from pyspark.sql.functions import col, isnan, isnull, when, sum as spark_sum

cols_to_check = ["Flow Bytes/s", "Flow Packets/s"]

stats_selected = df.select([
    spark_sum(
        when(
            isnan(col(f"`{c}`")) | isnull(col(f"`{c}`")) | (col(f"`{c}`") == float('inf')) | (col(f"`{c}`") == float('-inf')),
            1
        ).otherwise(0)
    ).alias(c)
    for c in cols_to_check
])

stats_selected.show(truncate=False)


+------------+--------------+
|Flow Bytes/s|Flow Packets/s|
+------------+--------------+
|4           |4             |
+------------+--------------+



In [ ]:
df = df.filter(
  ~(isnan(col("Flow Bytes/s")) | isnull(col("Flow Bytes/s")) | (col("Flow Bytes/s") == float('inf')) | (col("Flow Bytes/s") == float('-inf')) |
    isnan(col("Flow Packets/s")) | isnull(col("Flow Packets/s")) | (col("Flow Packets/s") == float('inf')) | (col("Flow Packets/s") == float('-inf')))
)


In [ ]:
from pyspark.sql.functions import col, isnan, isnull, when, sum as spark_sum

cols_to_check = ["Flow Bytes/s", "Flow Packets/s"]

stats_selected = df.select([
    spark_sum(
        when(
            isnan(col(f"`{c}`")) | isnull(col(f"`{c}`")) | (col(f"`{c}`") == float('inf')) | (col(f"`{c}`") == float('-inf')),
            1
        ).otherwise(0)
    ).alias(c)
    for c in cols_to_check
])

stats_selected.show(truncate=False)

+------------+--------------+
|Flow Bytes/s|Flow Packets/s|
+------------+--------------+
|0           |0             |
+------------+--------------+



In [ ]:
df = df.toDF(*[c.strip().replace(" ", "_") for c in df.columns])

In [ ]:
df.printSchema()

root
 |-- Destination_Port: integer (nullable = true)
 |-- Flow_Duration: integer (nullable = true)
 |-- Total_Fwd_Packets: integer (nullable = true)
 |-- Total_Backward_Packets: integer (nullable = true)
 |-- Total_Length_of_Fwd_Packets: integer (nullable = true)
 |-- Total_Length_of_Bwd_Packets: integer (nullable = true)
 |-- Fwd_Packet_Length_Max: integer (nullable = true)
 |-- Fwd_Packet_Length_Min: integer (nullable = true)
 |-- Fwd_Packet_Length_Mean: double (nullable = true)
 |-- Fwd_Packet_Length_Std: double (nullable = true)
 |-- Bwd_Packet_Length_Max: integer (nullable = true)
 |-- Bwd_Packet_Length_Min: integer (nullable = true)
 |-- Bwd_Packet_Length_Mean: double (nullable = true)
 |-- Bwd_Packet_Length_Std: double (nullable = true)
 |-- Flow_Bytes/s: double (nullable = true)
 |-- Flow_Packets/s: double (nullable = true)
 |-- Flow_IAT_Mean: double (nullable = true)
 |-- Flow_IAT_Std: double (nullable = true)
 |-- Flow_IAT_Max: integer (nullable = true)
 |-- Flow_IAT_Min: in

In [ ]:
# Sanitize column names: replace spaces, dots, and slashes
df = df.toDF(*[c.strip().replace(" ", "_").replace(".", "_").replace("/", "_") for c in df.columns])


In [ ]:
from collections import Counter
counts = Counter(df.columns)
dupes = [k for k, v in counts.items() if v > 1]
print("Duplicate column names:", dupes)


Duplicate column names: []


In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Label", outputCol="label_index")
df = indexer.fit(df).transform(df)

In [ ]:
df = df.withColumn("weight",
    when(col("Label") == "Benign", 0.6).
    when(col("Label") == "DoS", 1.2).
    when(col("Label") == "Web Attack", 2.0).
    otherwise(1.0)
)

In [ ]:
feature_cols = [c for c in df.columns if c not in ("Label", "label_index", "weight")]

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier

# Assume 'Label' is your target column and 'feature_cols' is already defined
indexer = StringIndexer(inputCol="Label", outputCol="label_index", handleInvalid="keep")
assembler = VectorAssembler(inputCols=feature_cols, outputCol="raw_features")
scaler = StandardScaler(inputCol="raw_features", outputCol="features", withStd=True, withMean=False)

rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label_index",
    weightCol="weight",  # Remove this line if you don’t have a 'weight' column
    numTrees=100
)

pipeline = Pipeline(stages=[indexer, assembler, scaler, rf])


In [ ]:
from pyspark.ml import PipelineModel

# Split the dataset
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)
train_df = train_df.drop("label_index")
test_df = test_df.drop("label_index")

# Fit the pipeline => returns a PipelineModel
pipeline_model: PipelineModel = pipeline.fit(train_df)  # ✅ explicitly typed as PipelineModel


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Transform the test dataset using the trained PipelineModel
predictions = pipeline_model.transform(test_df)

# Evaluate multiple metrics
metrics = ["accuracy", "f1", "weightedPrecision", "weightedRecall"]
for metric in metrics:
    evaluator = MulticlassClassificationEvaluator(
        labelCol="label_index", predictionCol="prediction", metricName=metric
    )
    score = evaluator.evaluate(predictions)
    print(f"{metric.capitalize()}: {score:.4f}")


NameError: name 'pipeline_model' is not defined

In [ ]:
# Save the trained PipelineModel
pipeline_model.save("/content/threat_detection_model_pipelinemodel")

# Zip the saved model directory for download
import shutil
shutil.make_archive("/content/threat_detection_model_pipelinemodel_zip", 'zip', "/content/threat_detection_model_pipelinemodel")


'/content/threat_detection_model_pipelinemodel_zip.zip'

In [ ]:
from google.colab import files
files.download("/content/threat_detection_model_pipelinemodel_zip.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>